In [ ]:
import json
import random

In [ ]:
def remove_indices(data, indieces):
    for del_idx in indieces:
        for row in data:
            del data[row][del_idx]

def disturb_dataset(data, img_per_class):
    
    for label_encoded in list(set(data['label_encoded'])):
        indices = [i for i, x in enumerate(data['label_encoded']) if x == label_encoded]
        if len(indices) > img_per_class:
            del_indices = random.sample(indices, len(indices) - img_per_class)
            del_indices.sort(reverse= True)
            remove_indices(data, del_indices)

In [ ]:
max_img_per_class_train = 56
max_img_per_class_test = 25

with open('dataset/data_train.json') as json_file:
    data_train = json.load(json_file)
    
with open('dataset/data_test.json') as json_file:
    data_test = json.load(json_file)

In [ ]:
print("before: {}".format(len(data_train['label'])))
disturb_dataset(data_train, max_img_per_class_train)
print("after: {}".format(len(data_train['label'])))

print("before: {}".format(len(data_test['label'])))
disturb_dataset(data_test, max_img_per_class_test)
print("after: {}".format(len(data_test['label'])))

In [ ]:
test = {k:0 for k in list(set(data_train['label_encoded']))}
for i in data_train['label_encoded']:
    test[i] += 1
{k: v for k, v in sorted(test.items(), key=lambda item: item[1])}

In [ ]:
path_to_images_folder = r'/home/fishial/Fishial/dataset/fishial_collection/data'
filenames = next(walk(path_to_images_folder), (None, None, []))[2]

In [ ]:
import cv2
from os import walk
import matplotlib.pyplot as plt

In [ ]:
data = cv2.imread(r'/home/fishial/Fishial/dataset/fishial_collection/data/00401597.jpg;')

In [ ]:
plt.imshow(data)

In [ ]:
import os
from os import listdir
from os.path import isfile, join
path_to_empty_ann = r'/home/fishial/Fishial/dataset/fishial_collection/data'
onlyfiles = [f for f in listdir(path_to_empty_ann) if isfile(join(path_to_empty_ann, f))]

In [ ]:
'00401597.jpg;' in onlyfiles

In [ ]:
import os
import sys
#Change path specificly to your directories
sys.path.insert(1, '/home/fishial/Fishial/Object-Detection-Model')
from module.segmentation_package.src.utils import remove_tmp_files
from module.segmentation_package.src.utils import read_json  
from module.segmentation_package.src.utils import save_json  
from module.segmentation_package.src.utils import get_mask
import numpy as np
import pyclipper

In [ ]:
folder_path = r'/home/fishial/Fishial/dataset/export'
json_path = r'03_export_Verified_ALL.json.json'

data = read_json(os.path.join(folder_path, json_path))
def get_image_by_id(data, id):
    for img_inst in data['images']:
        if img_inst['id'] == id:
            return img_inst
def fix_poly(poly, size):
    return 

In [ ]:
cnt = 0
for idx, ann in enumerate(data['annotations']):
    
    img = cv2.imread(r'/home/fishial/Fishial/dataset/fishial_collection/data/' + get_image_by_id(data, ann['image_id'])['file_name'] )
    try:
        poly_raw = ann['segmentation'][0]
    #         print(ann)
        height, width = img.shape[:2]
        polyline_main = [[ max(0, min(width, int(poly_raw[point_id * 2]) ) ), 
                          max(0, min(height, int(poly_raw[point_id * 2 + 1])))
                         ] for point_id in
                         range(int(len(poly_raw) / 2))]
        polygon = Polygon(polyline_main)
        if not polygon.is_valid:
            cnt += 1
            print(f"cnt: {cnt} idx: {idx}", end = '\r')
            data['annotations'][idx].update({'is_valid': False})
            mask = get_mask(img, np.array(polyline_main))
        else:
            data['annotations'][idx].update({'is_valid': True})
    except:
        data['annotations'][idx].update({'is_valid': False})
        

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

polygon = Polygon(polyline_main)
print(polygon.is_valid)

In [ ]:
save_json(data, os.path.join(folder_path, "fixed_all_json.json"))

In [ ]:
def is_valid(ann, img_folder = r'/home/fishial/Fishial/dataset/fishial_collection/data' ):
    img = cv2.imread(os.path.join(img_folder, get_image_by_id(data, ann['image_id'])['file_name'] ))
    poly_raw = ann['segmentation'][0]
    height, width = img.shape[:2]
    
    polyline_main = [[ max(0, min(width, int(poly_raw[point_id * 2]))), 
                      max(0, min(height, int(poly_raw[point_id * 2 + 1])))
                     ] for point_id in
                     range(int(len(poly_raw) / 2))]
    
    polygon = Polygon(polyline_main)
    return polygon.is_valid

In [1]:
import sys

# Change path specificly to your directories
sys.path.insert(1, '/home/fishial/Fishial/Object-Detection-Model')
from module.classification_package.src.dataset import FishialDataset, FishialDatasetOnlineCuting
from module.classification_package.src.dataset import BalancedBatchSampler
from torchvision import transforms
from PIL import Image

In [2]:
ds_train = FishialDatasetOnlineCuting(
        path_to_images_folder=r'/home/fishial/Fishial/dataset/fishial_collection/data',
        path_to_COCO_file=r'/home/fishial/Fishial/dataset/export/fixed_all_json.json',
        dataset_type='train',
        train_state=True,
        transform=transforms.Compose([transforms.Resize((224, 224), Image.BILINEAR),
                                      transforms.TrivialAugmentWide(),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(),
                                      transforms.RandomErasing(p=0.358, scale=(0.05, 0.4), ratio=(0.05, 6.1),
                                                               value=0, inplace=False),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    )

/home/fishial/miniconda3/envs/detectron2_env/lib/python3.6/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [3]:
balanced_batch_sampler_ds_train = BalancedBatchSampler(ds_train, 5, 15)

In [4]:
from torch import nn
from torch.utils.data import DataLoader

In [13]:

data_loader_train = DataLoader(ds_train, batch_sampler=balanced_batch_sampler_ds_train,
                               num_workers=2,
                               pin_memory=True)  # Construct your Dataloader here

In [15]:
for batch_idx, batch in enumerate(data_loader_train):
    print(f"len batch: {len(batch)}")
    batch = tuple(t.to('cuda') for t in batch)
    images, labels = batch
    print(f"len images: {len(images)}")
    print(f"len labels: {len(labels)}")
    print(10 * "+")

len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75
len labels: 75
++++++++++
len batch: 2
len images: 75


KeyboardInterrupt: 

In [ ]:
batch[0]

In [8]:
classes = next(iter(data_loader_train))   

In [12]:
classes[0][0].shape

torch.Size([3, 224, 224])